Related article: https://www.ivanlai.project-ds.net/post/conditional-text-generation-by-fine-tuning-gpt-2

Preprocessing code in [this](https://github.com/ivanlai/Conditional_Text_Generation) Github repository.

### Install and import libraries

In [1]:
%%time
%%capture
!pip install transformers

CPU times: user 43.2 ms, sys: 22.4 ms, total: 65.6 ms
Wall time: 6.27 s


In [37]:
import os
import io
import requests
import numpy as np
import pandas as pd
import re
import zipfile
import random
import time
import csv
import datetime
from itertools import compress
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from transformers import AutoTokenizer, AutoConfig, AutoModelForPreTraining, \
                         AdamW, get_linear_schedule_with_warmup, \
                         TrainingArguments, BeamScorer, Trainer, \
                         GPT2Tokenizer, GPT2LMHeadModel, AdamW

import torch
from torch.utils.data import Dataset, random_split, DataLoader, \
                             RandomSampler, SequentialSampler

from IPython.display import clear_output

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('You got a GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

print(f"PyTorch version: {torch.__version__}")

# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)

You got a GPU: Tesla V100-SXM2-16GB
PyTorch version: 1.9.0+cu102


In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Configurations

In [39]:
DEBUG           = False

INPUT_DIR       = 'articles'

USE_APEX        = True
APEX_OPT_LEVEL  = 'O1'

MODEL           = 'sberbank-ai/rugpt3small_based_on_gpt2' #{'gpt2', gpt2, gpt2-medium, gpt2-large, gpt2-xl}

UNFREEZE_LAST_N = 6 #The last N layers to unfreeze for training

SPECIAL_TOKENS  = { "bos_token": "<|BOS|>",
                    "eos_token": "<|EOS|>",
                    "unk_token": "<|UNK|>",                    
                    "pad_token": "<|PAD|>",
                    "sep_token": "<|SEP|>"}
                    
MAXLEN          = 768  #{768, 1024, 1280, 1600}

TRAIN_SIZE      = 0.8

if USE_APEX:
    TRAIN_BATCHSIZE = 4
    BATCH_UPDATE    = 24
else:
    TRAIN_BATCHSIZE = 4
    BATCH_UPDATE    = 16

EPOCHS          = 15
LR              = 5e-4
EPS             = 1e-8
WARMUP_STEPS    = 1e2

SEED            = 2021

In [40]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

## Preprocessing df and create dictionary for model

In [41]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/alco.xls')

In [42]:
df['ingridients'] = df['ingridients'].str.lower()
df['ingridients'] = df['ingridients'].str.split(' ,')
df.drop(['weight'], axis = 1, inplace = True)
df.drop(df.index[df['recipe'].str.find('настаивай') != -1], axis = 0, inplace = True)

df = df.reset_index()
df =  df.reindex(columns=['index', 'name', 'recipe', 'ingridients'])
df.drop('index', axis = 1, inplace=True)
df = df.reset_index()
df

,index,name,recipe,ingridients
0,0,Манхэттен,Налей в стакан для смешивания красный вермут 2...,"[бурбон, красный вермут, ангостура биттер, кок..."
1,1,Секс на пляже,"Наполни слинг кубиками льда доверху,Налей в ше...","[водка, персиковый ликер, клюквенный сок, анан..."
2,2,Космополитен,"Налей в шейкер лаймовый сок 10 мл, клюквенный ...","[цитрусовая водка, трипл сек, клюквенный сок, ..."
3,3,Б-52,"Налей в стопку кофейный ликер 15 мл,Используя ...","[кофейный ликер, айриш крим, трипл сек ]"
4,4,Голубая лагуна,"Наполни харрикейн кубиками льда доверху,Налей ...","[водка, ликер блю кюрасао, спрайт, ананас, лед..."
...,...,...,...,...
1145,1145,Карибский бриз,"Наполни хайбол кубиками льда доверху,Налей гре...","[грейпфрутовая водка, домашняя ванильная водка..."
1146,1146,Текила санрайз,"Наполни хайбол кубиками льда доверху,Налей гре...","[серебряная текила, гренадин, апельсиновый сок..."
1147,1147,Мохито,Положи в хайбол лайм 3 дольки и подави мадлеро...,"[белый ром, сахарный сироп, содовая, лайм, мят..."
1148,1148,Белый русский,"Наполни рокс кубиками льда доверху,Налей в бок...","[водка, кофейный ликер, нежирные сливки, лед в..."


In [43]:
df['recipe'] = df['recipe'].str.replace(r',+(?=[А-Я])', '. ')
df['recipe'] = df['recipe'].str.replace('1 к. ', '1 коктейльную ')
df['recipe'] = df['recipe'].str.replace(' к. ', ' коктейльных ')
df['recipe'] = df['recipe'].str.replace('без газа', '')
df['recipe'] = df['recipe'].str.replace('-', '')
df['recipe'] = df['recipe'].str.replace('\xa0', '')
df['recipe'] = df['recipe'].str.replace('\xad', '')



df['ingridients'] = df['ingridients'].replace('Вода без газа', 'Вода')
for i in range(df.shape[0]):
  for j in range(len(df['ingridients'][i])):
    df['ingridients'][i][j] = df['ingridients'][i][j].rstrip()
    df['ingridients'][i][j] = df['ingridients'][i][j].replace('-', '')
    df['ingridients'][i][j] = df['ingridients'][i][j].replace('\xa0', '')
    df['ingridients'][i][j] = df['ingridients'][i][j].replace('\xad', '')
  if 'Вода без газа' in df['ingridients'][i]:
    item = df['ingridients'][i].index('Вода без газа')
    df['ingridients'][i][item] = 'Вода'
df

,index,name,recipe,ingridients
0,0,Манхэттен,Налей в стакан для смешивания красный вермут 2...,"[бурбон, красный вермут, ангостура биттер, кок..."
1,1,Секс на пляже,Наполни слинг кубиками льда доверху. Налей в ш...,"[водка, персиковый ликер, клюквенный сок, анан..."
2,2,Космополитен,"Налей в шейкер лаймовый сок 10 мл, клюквенный ...","[цитрусовая водка, трипл сек, клюквенный сок, ..."
3,3,Б-52,Налей в стопку кофейный ликер 15 мл. Используя...,"[кофейный ликер, айриш крим, трипл сек]"
4,4,Голубая лагуна,Наполни харрикейн кубиками льда доверху. Налей...,"[водка, ликер блю кюрасао, спрайт, ананас, лед..."
...,...,...,...,...
1145,1145,Карибский бриз,Наполни хайбол кубиками льда доверху. Налей гр...,"[грейпфрутовая водка, домашняя ванильная водка..."
1146,1146,Текила санрайз,Наполни хайбол кубиками льда доверху. Налей гр...,"[серебряная текила, гренадин, апельсиновый сок..."
1147,1147,Мохито,Положи в хайбол лайм 3 дольки и подави мадлеро...,"[белый ром, сахарный сироп, содовая, лайм, мят..."
1148,1148,Белый русский,Наполни рокс кубиками льда доверху. Налей в бо...,"[водка, кофейный ликер, нежирные сливки, лед в..."


In [ ]:
#Создание уникальных значений ингредиентов
from pandas.core.common import flatten

unqiue_ingredients = set(flatten(df.ingridients))
unqiue_ingredients

In [ ]:
data = df.set_index('index').T.to_dict('list')
data

### Datasets and loaders

In [11]:
class myDataset(Dataset):

    def __init__(self, data, tokenizer, randomize=True):

        title, text, keywords = [], [], []
        for k, v in data.items():
            title.append(v[0])
            text.append(v[1])
            keywords.append(v[2])

        self.randomize = randomize
        self.tokenizer = tokenizer 
        self.title     = title
        self.text      = text
        self.keywords  = keywords  

    #---------------------------------------------#

    @staticmethod
    def join_keywords(keywords, randomize=True):
        N = len(keywords)

        #random sampling and shuffle
        if randomize: 
            M = random.choice(range(N+1))
            keywords = keywords[:M]
            random.shuffle(keywords)

        return ','.join(keywords)

    #---------------------------------------------#

    def __len__(self):
        return len(self.text)

    #---------------------------------------------#
    
    def __getitem__(self, i):
        keywords = self.keywords[i].copy()
        kw = self.join_keywords(keywords, self.randomize)
        
        input = SPECIAL_TOKENS['bos_token'] + self.title[i] + \
                SPECIAL_TOKENS['sep_token'] + kw + SPECIAL_TOKENS['sep_token'] + \
                self.text[i] + SPECIAL_TOKENS['eos_token']

        encodings_dict = self.tokenizer(input,                                   
                                   truncation=True, 
                                   max_length=MAXLEN, 
                                   padding="max_length")   
        
        input_ids = encodings_dict['input_ids']
        attention_mask = encodings_dict['attention_mask']
        
        return {'label': torch.tensor(input_ids),
                'input_ids': torch.tensor(input_ids), 
                'attention_mask': torch.tensor(attention_mask)}

In [12]:
def split_data(data, S=TRAIN_SIZE):
    # Shuffle ids
    ids = list(data.keys())
    random.shuffle(ids)

    # Split into training and validation sets    
    train_size = int(S * len(data))

    train_ids = ids[:train_size]
    val_ids = ids[train_size:]

    train_data = dict()
    for id in train_ids:
        train_data[id] = data[id]

    val_data = dict()
    for id in val_ids:
        val_data[id] = data[id]

    return train_data, val_data

### Loading Tokenizer, Config and Model

In [13]:
def get_tokenier(special_tokens=None):
    tokenizer = GPT2Tokenizer.from_pretrained(MODEL) #GPT2Tokenizer

    if special_tokens:
        tokenizer.add_special_tokens(special_tokens)
        print("Special tokens added")
    return tokenizer

def get_model(tokenizer, special_tokens=None, load_model_path=None):

    #GPT2LMHeadModel
    if special_tokens:
        config = AutoConfig.from_pretrained(MODEL, 
                                            bos_token_id=tokenizer.bos_token_id,
                                            eos_token_id=tokenizer.eos_token_id,
                                            sep_token_id=tokenizer.sep_token_id,
                                            pad_token_id=tokenizer.pad_token_id,
                                            output_hidden_states=False)
    else: 
        config = AutoConfig.from_pretrained(MODEL,                                     
                                            pad_token_id=tokenizer.eos_token_id,
                                            output_hidden_states=False)    

    #----------------------------------------------------------------#
    model = AutoModelForPreTraining.from_pretrained(MODEL, config=config)

    if special_tokens:
        #Special tokens added, model needs to be resized accordingly
        model.resize_token_embeddings(len(tokenizer))

    if load_model_path:
        model.load_state_dict(torch.load(load_model_path))

    model.cuda()
    return model

In [14]:
%%time

tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer, 
                  special_tokens=SPECIAL_TOKENS,
                 )


Special tokens added



CPU times: user 13.6 s, sys: 3.54 s, total: 17.1 s
Wall time: 24.9 s


In [15]:
# - Freeze selective layers:
# - Freeze all layers except last n:
for parameter in model.parameters():
    parameter.requires_grad = False

for i, m in enumerate(model.transformer.h):        
    #Only un-freeze the last n transformer blocks
    if i+1 > 12 - UNFREEZE_LAST_N:
        for parameter in m.parameters():
            parameter.requires_grad = True 

for parameter in model.transformer.ln_f.parameters():        
    parameter.requires_grad = True

for parameter in model.lm_head.parameters():        
    parameter.requires_grad = True

In [16]:
train_data, val_data = split_data(data)

train_dataset = myDataset(train_data, tokenizer)
val_dataset = myDataset(val_data, tokenizer, randomize=False)

f'There are {len(train_dataset) :,} samples for training, and {len(val_dataset) :,} samples for validation testing'

'There are 920 samples for training, and 230 samples for validation testing'

### Fine-tune GPT2 using Trainer

In [ ]:
%%time

training_args = TrainingArguments(
    output_dir="/content/",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=TRAIN_BATCHSIZE,
    per_device_eval_batch_size=TRAIN_BATCHSIZE,
    gradient_accumulation_steps=BATCH_UPDATE,
    evaluation_strategy="epoch",
    fp16=True,
    fp16_opt_level=APEX_OPT_LEVEL,
    warmup_steps=WARMUP_STEPS,    
    learning_rate=LR,
    adam_epsilon=EPS,
    weight_decay=0.01,        
    save_total_limit=1,
    load_best_model_at_end=True,     
)

#---------------------------------------------------#
trainer = Trainer(
    model=model,
    args=training_args,    
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

#---------------------------------------------------#
trainer.train()
trainer.save_model()    

In [19]:
# Save to G-Drive ----------------------------------#
!cp -r 'pytorch_model.bin' '/content/pytorch_model_recipe.bin'

### Сохранение модели для генерации рецепта



In [20]:
# !cp -r '/content/pytorch_model_V2.bin' 'pytorch_model.bin' 

In [36]:
SAVE_DIR = '/content/drive/MyDrive/Colab Notebooks/titles'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'ALCO.pt')
torch.save(model.state_dict(),  MODEL_SAVE_PATH )

In [22]:
model.save_pretrained('/content/sample_data')

Configuration saved in /content/sample_data/config.json
Model weights saved in /content/sample_data/pytorch_model.bin


## Генерация рецептов

In [ ]:
tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer, 
                  special_tokens=SPECIAL_TOKENS,
                  load_model_path='pytorch_model_recipe.bin')

In [33]:
title = ""
keywords = ['малина','ликер', 'водка', 'пиво', 'вишня']
kw = myDataset.join_keywords(keywords, randomize=False)

prompt = SPECIAL_TOKENS['bos_token'] + title + \
         SPECIAL_TOKENS['sep_token'] + kw + SPECIAL_TOKENS['sep_token']
         
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)

model.eval();

In [ ]:
# Top-p (nucleus) text generation (10 samples):
sample_outputs = model.generate(generated, 
                                do_sample=True,   
                                min_length=50, 
                                max_length=MAXLEN,
                                top_k=30,                                 
                                top_p=0.7,        
                                temperature=0.6,
                                repetition_penalty=2.0,
                                num_return_sequences=100
                                )
recipes = []

for i, sample_output in enumerate(sample_outputs):
    text = tokenizer.decode(sample_output, skip_special_tokens=True)
    a = len(title) + len(','.join(keywords))   
    recipes.append("{}".format(text[a:])) 
    print("{}\n\n".format(text[a:]))

In [31]:
# Создание словаря с подсчётом слов, которые есть в ввёднных ингредиентах
actual_recipe_dict = {}
for recipe in recipes:
  for i in keywords:
    if recipe.find(i.lower()[:-2]) != -1:
      if recipe in actual_recipe_dict:
        actual_recipe_dict[recipe] += 1
      else:
        actual_recipe_dict[recipe] = 1

actual_recipe_dict
sorted_tuple = sorted(actual_recipe_dict.items(), key=lambda x: -x[1])
# При условии, что все три ингридиента. А если не три, то топ 3. 
dict(sorted_tuple)
sorted_tuple[0][0]

'Наполни рокс кубиками льда доверху. Налей ликер мараскино 20 мл и водку 50 г. Долей клюквенный сок 100 дэш в бокал для вина на шпажке'

In [35]:
# Beam-search text generation:
sample_outputs = model.generate(generated, 
                                do_sample=True,   
                                max_length=MAXLEN,                                                      
                                num_beams=5,
                                repetition_penalty=5.0,
                                early_stopping=True,      
                                num_return_sequences=10
                                )

for i, sample_output in enumerate(sample_outputs):
    text = tokenizer.decode(sample_output, skip_special_tokens=True)
    a = len(title) + len(','.join(keywords))    
    print("{}\n\n".format(text[a:]))

Наполни рокс кубиками льда доверху. Положи в шейкер вишневый джем 4 коктейльных ложки и подави мадлером. Налей лимонный сок 15 мл, ликер трипл сек 20 мл и водку 40 мл. Наполни шейкер кубиками льда и взбей. Перелей через стрейнер и ситечко в рокс. Укрась веточкой мяты


Наполни рокс кубиками льда доверху. Налей в шейкер лаймовый сок 15 мл, ликер мараскино 20 мл и водку 40 мл. Наполни шейкер кубиками льда и взбей. Перелей через стрейнер в рокс. Укрась апельсиновой цедрой


Налей в стопку вишневый ликер 20 мл. Используя коктейльную ложку, уложи слой водки 20 мл


Наполни хайбол кубиками льда доверху. Налей в шейкер лаймовый сок 15 мл, ликер мараскино 20 мл и водку 40 мл. Наполни шейкер кубиками льда и взбей. Перелей через стрейнер в хайбол. Укрась кружком лайма


Наполни рокс кубиками льда доверху. Положи в шейкер вишневый джем 4 коктейльных ложки и подави мадлером. Налей лимонный сок 15 мл, ликер трипл сек 20 мл и водку 40 мл. Наполни шейкер кубиками льда и взбей. Перелей через стрейнер 

## Создание словаря и обучение модели для генерации названия

In [ ]:
df_title = df.copy()

In [ ]:
df_title = df_title.reindex(columns=['index', 'recipe', 'name',  'ingridients'])

In [ ]:
df_title.head(10)

,index,recipe,name,ingridients
0,0,Налей в стакан для смешивания красный вермут 2...,Манхэттен,"[бурбон, красный вермут, ангостура биттер, кок..."
1,1,Наполни слинг кубиками льда доверху. Налей в ш...,Секс на пляже,"[водка, персиковый ликер, клюквенный сок, анан..."
2,2,"Налей в шейкер лаймовый сок 10 мл, клюквенный ...",Космополитен,"[цитрусовая водка, трипл сек, клюквенный сок, ..."
3,3,Налей в стопку кофейный ликер 15 мл. Используя...,Б-52,"[кофейный ликер, айриш крим, трипл сек]"
4,4,Наполни харрикейн кубиками льда доверху. Налей...,Голубая лагуна,"[водка, ликер блю кюрасао, спрайт, ананас, лед..."
5,5,Сделай на бокале для маргариты соленую окаемку...,Маргарита,"[серебряная текила, трипл сек, сахарный сироп,..."
6,6,Наполни бокал для вина льдом. Налей в бокал пр...,Апероль Шприц,"[апероль, просекко, содовая, апельсин, лед в к..."
7,7,Наполни френчпресс кубиками льда доверху. Нале...,Лонг айленд айс ти,"[лондонский сухой джин, водка, белый ром, сере..."
8,8,Наполни рокс кубиками льда доверху. Налей в ше...,Роза Багси,"[водка, сироп розы, лаймовый кордиал, лаймовый..."
9,9,Налей в стопку водку 50 мл. Налей в другую сто...,Малевич шутер,"[водка, соленый огурец, мед]"


In [ ]:
data_title = df_title.set_index('index').T.to_dict('list')
data_title

In [ ]:
%%time

tokenizer_title = get_tokenier(special_tokens=SPECIAL_TOKENS)
model_title = get_model(tokenizer_title, 
                  special_tokens=SPECIAL_TOKENS,
                 )

loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/1b36eeb1fd7b3a6ec11bf46bde2c38e7e68f71ec774694b9e886c86001aab35d.c483bc3440d25937fdac74506b73b76ee6e67f778a804756214363fc2a1a66ef
loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/479aa59074c4dcd4c36106252da033d03bc92e3010947ce1d3714de224c2af1f.7362c0dbb32f750eeea5a5b93bbd0c6876eac41453369265d5a49df1c9142b6f
loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/tokenizer_config.json from cache at None
loading file https://huggingface.co/sberbank-ai/rugpt3smal

Special tokens added


loading configuration file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/06f48b6b3173390d047e15d691fda67ae4ea7733a5eea4b6e0115f5099c4e700.b5cdfa39c63384f94159c36bc9042660c747cea5cf520b43d543bd2c68b3164d
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50257,
  "embd_pdrop": 0.1,
  "eos_token_id": 50258,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "pad_token_id": 50259,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "sep_token_id": 50260,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_v

CPU times: user 2.61 s, sys: 317 ms, total: 2.93 s
Wall time: 8.54 s


In [ ]:
train_data_title, val_data_title = split_data(data_title)

train_dataset_title = myDataset(train_data_title, tokenizer_title)
val_dataset_title = myDataset(val_data_title, tokenizer_title, randomize=False)

f'There are {len(train_dataset_title) :,} samples for training, and {len(val_dataset_title) :,} samples for validation testing'

'There are 920 samples for training, and 230 samples for validation testing'

In [ ]:
%%time

training_args_title = TrainingArguments(
    output_dir="/content/",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=TRAIN_BATCHSIZE,
    per_device_eval_batch_size=TRAIN_BATCHSIZE,
    gradient_accumulation_steps=BATCH_UPDATE,
    evaluation_strategy="epoch",
    fp16=True,
    fp16_opt_level=APEX_OPT_LEVEL,
    warmup_steps=WARMUP_STEPS,    
    learning_rate=LR,
    adam_epsilon=EPS,
    weight_decay=0.01,        
    save_total_limit=1,
    load_best_model_at_end=True,     
)

#---------------------------------------------------#
trainer_title = Trainer(
    model=model_title,
    args=training_args_title,    
    train_dataset=train_dataset_title,
    eval_dataset=val_dataset_title,
    tokenizer=tokenizer_title
)

#---------------------------------------------------#
trainer_title.train()
trainer_title.save_model()    

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp fp16 backend
***** Running training *****
  Num examples = 920
  Num Epochs = 15
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 24
  Total optimization steps = 135


Epoch,Training Loss,Validation Loss
0,No log,9.439652
1,No log,1.406001
2,No log,0.415639
3,No log,0.269523
4,No log,0.208465
5,No log,0.177891
6,No log,0.167347
7,No log,0.159580
8,No log,0.159949
9,No log,0.158882


***** Running Evaluation *****
  Num examples = 230
  Batch size = 4
Saving model checkpoint to /content/checkpoint-9
Configuration saved in /content/checkpoint-9/config.json
Model weights saved in /content/checkpoint-9/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-9/tokenizer_config.json
Special tokens file saved in /content/checkpoint-9/special_tokens_map.json
added tokens file saved in /content/checkpoint-9/added_tokens.json
Deleting older checkpoint [/content/checkpoint-135] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 230
  Batch size = 4
Saving model checkpoint to /content/checkpoint-18
Configuration saved in /content/checkpoint-18/config.json
Model weights saved in /content/checkpoint-18/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-18/tokenizer_config.json
Special tokens file saved in /content/checkpoint-18/special_tokens_map.json
added tokens file saved in /content/checkpoint-18/added_tokens.json
Del

Epoch,Training Loss,Validation Loss
0,No log,9.439652
1,No log,1.406001
2,No log,0.415639
3,No log,0.269523
4,No log,0.208465
5,No log,0.177891
6,No log,0.167347
7,No log,0.159580
8,No log,0.159949
9,No log,0.158882


Saving model checkpoint to /content/
Configuration saved in /content/config.json
Model weights saved in /content/pytorch_model.bin
tokenizer config file saved in /content/tokenizer_config.json
Special tokens file saved in /content/special_tokens_map.json
added tokens file saved in /content/added_tokens.json


CPU times: user 31min 2s, sys: 4min 11s, total: 35min 14s
Wall time: 35min 51s


In [ ]:
SAVE_DIR = '/content/'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'ALCO_title.pt')
torch.save(model_title.state_dict(),  MODEL_SAVE_PATH )

In [ ]:
# title = ""
# recipe = "Наполни хайбол кубиками льда доверху. Налей клубничный сироп 30 мл и ревеневый ликер 5 мл. Добавь односолодовый виски из Хайленда 30 мл. Долей содовую доверху и аккуратно размешай коктейльной ложкой. Укрась одной крупно нарезанной клубникой"
kw = myDataset.join_keywords(keywords, randomize=False)
recipe = sorted_tuple[0][0]

prompt = SPECIAL_TOKENS['bos_token'] + recipe + \
         SPECIAL_TOKENS['sep_token'] + kw + SPECIAL_TOKENS['sep_token']
         
generated_title = torch.tensor(tokenizer_title.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated_title = generated_title.to(device)

model_title.eval();

In [ ]:
# Top-p (nucleus) text generation (10 samples):
sample_outputs_title = model_title.generate(generated_title, 
                                do_sample=True,   
                                min_length=50, 
                                max_length=MAXLEN,
                                top_k=30,                                 
                                top_p=0.7,        
                                temperature=0.93,
                                repetition_penalty=2.0,
                                num_return_sequences=100
                                )
titles = []
for i, sample_output in enumerate(sample_outputs_title):
    text = tokenizer_title.decode(sample_output, skip_special_tokens=True)
    a = len(recipe) + len(','.join(keywords))   
    titles.append("{}".format(text[a:])) 
    print("{}\n\n".format(text[a:]))

In [ ]:
set_title = set(titles)

In [ ]:
import random

print()

['44: Водяная зараза']


##Выдача результатов готовых рецептов

In [44]:
df_result = df.copy()
df_result.ingridients = df_result.ingridients.apply(', '.join)

In [45]:
df_result.to_csv('ready_recipe.csv')

In [49]:
df_result = pd.read_csv('ready_recipe.csv')
df_result.drop('Unnamed: 0', axis = 1, inplace=True)
df_result

,index,name,recipe,ingridients
0,0,Манхэттен,Налей в стакан для смешивания красный вермут 2...,"бурбон, красный вермут, ангостура биттер, кокт..."
1,1,Секс на пляже,Наполни слинг кубиками льда доверху. Налей в ш...,"водка, персиковый ликер, клюквенный сок, анана..."
2,2,Космополитен,"Налей в шейкер лаймовый сок 10 мл, клюквенный ...","цитрусовая водка, трипл сек, клюквенный сок, л..."
3,3,Б-52,Налей в стопку кофейный ликер 15 мл. Используя...,"кофейный ликер, айриш крим, трипл сек"
4,4,Голубая лагуна,Наполни харрикейн кубиками льда доверху. Налей...,"водка, ликер блю кюрасао, спрайт, ананас, лед ..."
...,...,...,...,...
1145,1145,Карибский бриз,Наполни хайбол кубиками льда доверху. Налей гр...,"грейпфрутовая водка, домашняя ванильная водка,..."
1146,1146,Текила санрайз,Наполни хайбол кубиками льда доверху. Налей гр...,"серебряная текила, гренадин, апельсиновый сок,..."
1147,1147,Мохито,Положи в хайбол лайм 3 дольки и подави мадлеро...,"белый ром, сахарный сироп, содовая, лайм, мята..."
1148,1148,Белый русский,Наполни рокс кубиками льда доверху. Налей в бо...,"водка, кофейный ликер, нежирные сливки, лед в ..."


In [30]:
%%time
ready_recipe_dict = {}

for index, row in df_result.iterrows():
  for keyword in keywords:
    if row.ingridients.find(keyword) != -1:
      if index in ready_recipe_dict:
        ready_recipe_dict[index] += 1
      else:
        ready_recipe_dict[index] = 1

max_ready_dict = max(ready_recipe_dict.values())

for k, v in ready_recipe_dict.items():
        if v == max_ready_dict:
          print(f'{df_result["name"][k]} : {df_result["recipe"][k]}')

Секс на пляже : Наполни слинг кубиками льда доверху. Налей в шейкер клюквенный сок 40 мл, ананасовый сок 40 мл, персиковый ликер 25 мл и водку 50 мл. Наполни шейкер кубиками льда и взбей. Перелей через стрейнер в слинг. Укрась кусочком ананаса и ягодой малины на коктейльной шпажке
Морс мартини : Положи в шейкер клубнику 2 ягоды, малину 2 ягоды, чернику 1 коктейльную ложку и подави мадлером. Налей клюквенный сок 30 мл, сахарный сироп 10 мл, черносмородиновый ликер 10 мл и водку 30 мл. Наполни шейкер кубиками льда и взбей. Перелей через стрейнер и ситечко в охлажденный коктейльный бокал. Укрась веточкой красной смородины
Белый и перечный : Наполни слинг кубиками льда доверху. Положи в шейкер пюре личи 12 коктейльных ложек. Налей белок перепелиного яйца, лимонный сок 30 мл, сахарный сироп 10 мл, ликер личи 20 мл и водку 40 мл. Наполни шейкер кубиками льда и тщательно взбей. Перелей через стрейнер в слинг. Укрась очищенным личи, малинкой и щепоткой черного молотого перца
Блинк-блинк : Нале

### Generating text with raw GPT2

In [ ]:
tokenizer_raw = get_tokenier()
model_raw = get_model(tokenizer_raw)

loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/1b36eeb1fd7b3a6ec11bf46bde2c38e7e68f71ec774694b9e886c86001aab35d.c483bc3440d25937fdac74506b73b76ee6e67f778a804756214363fc2a1a66ef
loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/479aa59074c4dcd4c36106252da033d03bc92e3010947ce1d3714de224c2af1f.7362c0dbb32f750eeea5a5b93bbd0c6876eac41453369265d5a49df1c9142b6f
loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/tokenizer_config.json from cache at None
loading file https://huggingface.co/sberbank-ai/rugpt3smal

In [ ]:
prompt = 'Наше прошлое скакало по окну'

generated = torch.tensor(tokenizer_raw.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)

model_raw.eval()
sample_outputs = model_raw.generate(generated, 
                                do_sample=True,   
                                max_length=MAXLEN,                                                      
                                num_beams=5,
                                repetition_penalty=5.0,
                                early_stopping=True,      
                                num_return_sequences=1
                                )

for i, sample_output in enumerate(sample_outputs):
    print("{}: {}\n\n".format(i, tokenizer_raw.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Наше прошлое скакало по окну,
		А наше будущее прыгало в окно.
		Мы мечтали о том, что будет когда-нибудь,
		Но все это было не так уж и важно.
		И вот наступил тот день, когда мы поняли,
		Что нам никогда не суждено быть вместе.
		В этот день я понял, что мне ничего не светит,
		Я знал, что у меня нет будущего,
		Оно осталось где-то там, за горизонтом.
		Мне казалось, что сейчас наступит конец света,
		Ведь с тех пор прошло уже много лет,
		С тех пор как умерла моя мама…
		Все эти годы я думал только об одном:
		Как бы поскорее избавиться от этого кошмара.
		Забыть обо всем на свете!
		Уйти из этой жизни навсегда!
		Отказаться от всех земных радостей!
		Вернуться к нормальной жизни!
		Посмотреть на мир другими глазами!
		Не думать ни о чем плохом!
		Пожелать друг другу всего самого лучшего!
		Быть счастливыми вечно!
		Жить полной жизнью!
		Да здравствует любовь!
		Будьте счастливы всегда!
		Пусть каждый день приносит вам лишь радость!
		Поздравляю вас с наступающим Новым годом!
		Д